# Анализ времени ответов операторов службы поддержки


Данный анализ направлен на оценку эффективности работы операторов службы поддержки путем измерения времени их ответов на обращения клиентов. Анализ учитывает рабочее время операторов (с 9:30) и различные метрики производительности.

<h2>Панель анализа данных -- <a style="margin: 0.1rem auto;color: #8FA0FF;
            text-decoration: underline;" href="https://responsive-mn2dzbwzxyw86psk87oj8n.streamlit.app/" target="_blank">Dashboard</a></h2>
<div class="container" style="
             max-width: 87%;
            margin: 20px auto;
            background-color: #fff9c4;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            ">
    <div style=" margin: 1rem auto;
            max-width: 35rem;
            border-radius: 8px;
            position: relative;
        paddihg-top: 1rem;
            text-align: center;" class="note-column"> <!-- добавлено text-align: center -->
        <div style="background-color: #fff9c4;
            " class="link-column"> <!-- изменено для центровки -->
        </div><p style="">Для выполнения задания я выбрала альтернативное решение с использованием Python и HTML, так как этот стек предоставляет более гибкие возможности для анализа данных и визуализации. Реализованное решение включает все необходимые расчеты и интерактивные элементы, позволяя легко масштабировать и модифицировать дашборд под новые задачи. Готова продемонстрировать работу приложения и обсудить технические детали реализации.</p>
    </div>
</div>

## SQL-запрос предоставляет подробную статистику по времени ответа каждого менеджера:

1. **Первый CTE (message_blocks):**
   - Группирует сообщения по диалогам (entity_id)
   - Использует LAG для определения предыдущих сообщений
   - Конвертирует Unix timestamp в нормальную дату/время
   - Сортирует сообщения по времени создания

2. **Второй CTE (response_times):** 
   - Вычисляет время ответа для каждого сообщения 
   - Учитывает только пары "входящее-исходящее" сообщение 
   - Обрабатывает особые случаи:
      - Сообщения в нерабочее время (00:00-09:30)
      - Сообщения в рабочее время (09:30-00:00) 
   - Присоединяет информацию о менеджерах

3. **Финальный SELECT:**
   - Группирует результаты по менеджерам
   - Вычисляет статистику:
        - Общее количество ответов
        - Среднее время ответа
        -  Минимальное время ответа
        -  Максимальное время ответа
   - Сортирует по среднему времени ответа

#### Обработка блоков сообщений:
- Использует LAG для определения предыдущего сообщения
- Учитывает только первое сообщение из каждого блока

#### Учет рабочего времени:
- Корректно обрабатывает период 09:30-00:00
- Для сообщений, пришедших ночью, отсчет начинается с 09:30

#### Точность расчетов:
- Время конвертируется в минуты
- Результаты округляются до 2 знаков после запятой






 <h4>Запрос</h4>
    <pre style="background-color: #f8f8f8;
            border: 1px solid #ddd;
            border-radius: 4px;
            padding: 15px;
            overflow-x: auto;
            font-size: 14px;
            line-height: 1.4;"><code style="color: #333;
            font-family: 'Consolas', 'Monaco', monospace;">
WITH message_blocks AS (
    -- Группируем сообщения и определяем предыдущие сообщения в диалоге
    SELECT 
        message_id,
        type,
        entity_id,
        created_by,
        created_at,
        to_timestamp(created_at) AT TIME ZONE 'UTC' as created_at_ts,
        LAG(type) OVER (PARTITION BY entity_id ORDER BY created_at) as prev_type,
        LAG(created_at) OVER (PARTITION BY entity_id ORDER BY created_at) as prev_created_at,
        LAG(to_timestamp(created_at) AT TIME ZONE 'UTC') OVER (
            PARTITION BY entity_id ORDER BY created_at
        ) as prev_created_at_ts
    FROM test.chat_messages
),

response_times AS (
    -- Вычисляем время ответа для каждого сообщения
    SELECT 
        m.*,
        mg.name_mop,
        CASE 
            WHEN m.type = 'outgoing_chat_message' 
            AND m.prev_type = 'incoming_chat_message' THEN
                CASE 
                    -- Если предыдущее сообщение пришло в нерабочее время (00:00-09:30)
                    WHEN EXTRACT(HOUR FROM m.prev_created_at_ts) < 9 
                        OR (EXTRACT(HOUR FROM m.prev_created_at_ts) = 9 
                            AND EXTRACT(MINUTE FROM m.prev_created_at_ts) < 30)
                    THEN 
                        -- Считаем время от 09:30 того же дня
                        EXTRACT(EPOCH FROM (
                            m.created_at_ts - 
                            (date_trunc('day', m.prev_created_at_ts) + interval '9 hours 30 minutes')
                        ))/60
                    -- Для сообщений в рабочее время
                    WHEN EXTRACT(HOUR FROM m.created_at_ts) >= 9 
                        AND (EXTRACT(HOUR FROM m.created_at_ts) < 24)
                    THEN 
                        EXTRACT(EPOCH FROM (m.created_at_ts - m.prev_created_at_ts))/60
                    ELSE NULL
                END
            ELSE NULL
        END as response_time_minutes
    FROM message_blocks m
    LEFT JOIN test.managers mg ON m.created_by = mg.mop_id
)

-- Формируем итоговую статистику по каждому менеджеру
SELECT 
    name_mop as "Менеджер",
    COUNT(*) as "Количество ответов",
    ROUND(AVG(response_time_minutes)::numeric, 2) as "Среднее время ответа (мин)",
    ROUND(MIN(response_time_minutes)::numeric, 2) as "Минимальное время ответа (мин)",
    ROUND(MAX(response_time_minutes)::numeric, 2) as "Максимальное время ответа (мин)"
FROM response_times
WHERE response_time_minutes IS NOT NULL
AND name_mop IS NOT NULL
GROUP BY name_mop
ORDER BY "Среднее время ответа (мин)";</code></pre>

### Анализ данных - Python

In [7]:
# импорт библиотек
import pandas as pd
import psycopg2
from datetime import datetime, time
import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore")

In [ ]:
# Устанавливаем соединение с сервером
def connect_to_db():
    """Установка соединения"""
    try:
        conn = psycopg2.connect(
            host="rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net",
            port="6432",
            database="db1",
            user="test_user",
            password="j2M{CnnFq@"
        )
        return conn
    except Exception as e:
        print(f"Database connection error: {str(e)}")
        raise

In [ ]:
# получаем данные
def get_response_times():
    """Получаем время ответа оператора"""
    query = """
    WITH ranked_messages AS (
        SELECT 
            m.message_id,
            m.type,
            m.entity_id,
            m.created_by,
            m.created_at,
            mg.name_mop,
            LAG(m.type) OVER (PARTITION BY m.entity_id ORDER BY m.created_at) as prev_type,
            LAG(m.created_at) OVER (PARTITION BY m.entity_id ORDER BY m.created_at) as prev_created_at
        FROM test.chat_messages m
        LEFT JOIN test.managers mg ON m.created_by = mg.mop_id
        ORDER BY m.entity_id, m.created_at
    ),
    response_times AS (
        SELECT 
            rm.*,
            CASE 
                WHEN rm.type = 'outgoing_chat_message' 
                AND rm.prev_type = 'incoming_chat_message' THEN
                    CASE 
                        -- If previous message was during non-working hours (00:00-09:30)
                        WHEN TO_TIMESTAMP(rm.prev_created_at)::time < TIME '09:30:00' 
                        AND TO_TIMESTAMP(rm.created_at)::time >= TIME '09:30:00' THEN
                            EXTRACT(EPOCH FROM (
                                TO_TIMESTAMP(rm.created_at) - 
                                TO_TIMESTAMP(rm.prev_created_at)::date + INTERVAL '9 hours 30 minutes'
                            ))
                        -- If response is during working hours
                        WHEN TO_TIMESTAMP(rm.created_at)::time >= TIME '09:30:00' 
                        AND TO_TIMESTAMP(rm.created_at)::time < TIME '24:00:00' THEN
                            EXTRACT(EPOCH FROM (
                                TO_TIMESTAMP(rm.created_at) - 
                                TO_TIMESTAMP(rm.prev_created_at)
                            ))
                        ELSE NULL
                    END
                ELSE NULL
            END as response_time_seconds
        FROM ranked_messages rm
    )
    SELECT 
        name_mop,
        COUNT(*) as total_responses,
        ROUND(AVG(response_time_seconds)/60, 2) as avg_response_time_minutes,
        ROUND(MIN(response_time_seconds)/60, 2) as min_response_time_minutes,
        ROUND(MAX(response_time_seconds)/60, 2) as max_response_time_minutes
    FROM response_times
    WHERE response_time_seconds IS NOT NULL
    AND name_mop IS NOT NULL
    GROUP BY name_mop
    ORDER BY avg_response_time_minutes;
    """
    
    try:
        conn = connect_to_db()
        if conn:
            df = pd.read_sql_query(query, conn)
            conn.close()
            return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [ ]:
# формат результатов
def format_results(df):
    """Форматируем результаты"""
    if df is not None:
        # Create pivot table
        pivot_df = df.set_index('name_mop').T
        # Переводим имена колонок
        value_names = {
                    'total_responses': 'Всего ответов',
                    'avg_response_time_minutes': 'Среднее время ответа (мин)',
                    'min_response_time_minutes': 'Минимальное время ответа (мин)',
                    'max_response_time_minutes': 'Максимальное время ответа (мин)'
                }
        pivot_df.index = pivot_df.index.map(value_names)
        # Set column and index names
        pivot_df.columns.name = 'Персона'
        pivot_df.index.name = 'Значение'
        
        return pivot_df.T
    return None

In [143]:
# запускаем программу
if __name__ == "__main__":
    # получаем данные
    df = get_response_times()
    # формат отображения данных
    results = format_results(df)
   
    if results is not None:
        display(HTML('<h1>Анализ времени ответов операторов:</h1>'))
        display(results)
        
        # сохраняем файл CSV
        results.to_csv('response_times_analysis.csv', encoding='utf-8-sig')
        print("\nРезультаты сохранены в 'response_times_analysis.csv'")

Значение,Всего ответов,Среднее время ответа (мин),Минимальное время ответа (мин),Максимальное время ответа (мин)
Персона,,,,
Влада и Настя,46.0,4.71,0.42,20.12
Ангелина Милованова,61.0,6.10,0.03,38.58
Порхачева Полина,14.0,8.37,0.43,23.60
Полина Мирзоян,48.0,10.43,0.23,95.22
Даша и Даша,150.0,11.56,0.17,1145.18
Настя и Даша,324.0,17.59,0.03,1144.27
Лиза и Ева,686.0,22.57,0.03,1154.35
Настя и Саша,784.0,30.54,0.03,1168.65
Ксюша и Джамиля,272.0,31.15,0.10,1901.08



Результаты сохранены в 'response_times_analysis.csv'
